In [1]:
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from datasets import load_dataset
import evaluate

import dotenv
import os
import shutil
import tqdm
import time
import json

dotenv.load_dotenv()
api_key = os.getenv("API_KEY")

Settings.llm = OpenAI(temperature=0.2, model="gpt-4")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

## Indexing Functions


In [ ]:
def chunk_text(text, chunk_size=500):
    """
    Split the document text into chunks based on a fixed character limit.

    Args:
        text (str): The entire document text.
        chunk_size (int): The maximum number of characters in each chunk.

    Returns:
        list[str]: A list of text chunks.
    """
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        if current_length + len(word) > chunk_size and current_chunk:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word)
        else:
            current_chunk.append(word)
            current_length += len(word) + 1  # Adding 1 for the space.

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def summarize_chunks(chunks, chunk_size=500):
    """
    Generate a summary for each text chunk using GPT-4.

    Args:
        chunks (list[str]): A list of text chunks to be summarized.

    Returns:
        list[str]: A list of summaries for each chunk.
    """
    summaries = []
    for chunk in chunks:
        prompt = f"Summarize the following text conceptually:\n\n{chunk}"
        response = Settings.llm.complete(prompt).text
        summaries.append(response.strip())

    #open summary save file name having chunk size in name
    with open(f"summaries_file_{chunk_size}.txt", 'w', encoding='utf-8') as file:
        for summary in summaries:
            file.write(summary + '\n')  # Each summary on a new line

    return summaries

def load_summaries(summaries_file='summaries_file_500.txt'):
    """
    Load summaries from a file where each summary is saved on a new line.

    Args:
        summaries_file (str): The file path from which to load summaries.

    Returns:
        list[str]: A list of summaries loaded from the file.
    """
    with open(summaries_file, 'r', encoding='utf-8') as file:
        summaries = [line.strip() for line in file if line.strip()]  # Exclude empty lines
    return summaries


In [ ]:
def create_index_from_text_with_ids(text, title, chunk_size=500):
    """
    Create an index from context, assigning a unique ID to each text chunk without including the ID in the text used for embedding.

    Args:
        text (str): The context to index.
        title (str): The title of the index for access later.
        chunk_size (int): The maximum number of characters in each chunk.

    Returns:
        VectorStoreIndex: The index created from the context, with IDs assigned to each text chunk.
    """
    chunks = chunk_text(text, chunk_size=chunk_size)
    index = get_index_by_title(title)

    if index is None:
        index = VectorStoreIndex()
        for i, chunk in enumerate(chunks):
            # Generate embedding for the chunk without the ID in the text
            embedding = Settings.embed_model.embed(chunk)
            
            # Assign a unique ID to the chunk's embedding
            chunk_id = f"text_chunk_{i}"
            
            # Add the embedding to the index with the unique ID, not including the ID in the text
            index.add_document(chunk_id, embedding)

        index.set_index_id(title)
        index.storage_context.persist()

    return index


def add_summarization_embeddings_with_ids(index, chunk_size=500, new_storage_location="storage_same_chunk_summary", index_title="summary_index"):
    """
    Generate embeddings for text summaries and add them to the existing vector store, assigning unique IDs to summaries without including the ID in the text for embedding.

    Args:
        index (VectorStoreIndex): The existing vector store index.
        summaries (list[str]): A list of text summaries to be embedded and added.

    Returns:
        VectorStoreIndex: The updated index with added summary embeddings and their unique IDs.
    
    """

    chunks = chunk_text(summaries, chunk_size=chunk_size)
    summaries = summarize_chunks(chunks, chunk_size=chunk_size)
    
    base_id = len(index)  # Starting point for new IDs
    title = index.index_id + index_title
    index_temp = get_index_by_title(title)
    if index_temp is not None:
        return index_temp
    
    for i, summary in enumerate(summaries):
        # Generate embedding for the summary without the ID in the text
        embedding = Settings.embed_model.embed(summary)
        
        # Assign a unique ID to the summary's embedding
        summary_id = f"summary_{base_id + i}"
        
        # Add the embedding to the index with the unique ID, not including the ID in the text
        index.add_document(summary_id, embedding)
    
    index.set_index_id(title)
    new_storage_context = StorageContext.from_defaults(persist_dir=new_storage_location)
    index.storage_context = new_storage_context  # Update the index's storage context to the new location
    index.storage_context.persist()  # Persist the index in the new location
    return index


In [3]:
def create_index_from_text(text, title):
    """
    Create an index from a piece of context. If an index with the given title already exists, it will be returned.

    Args:
        text (str): The context to index
        title (str): The title of the index for access later

    Returns:
        VectorStoreIndex: The index created from the context
    """
    index = get_index_by_title(title)
    if index is None:
        os.makedirs("tmp", exist_ok=True)
        with open("tmp/tmp.txt", "w") as f:
            f.write(text)
        documents = SimpleDirectoryReader("tmp").load_data()
        index = VectorStoreIndex.from_documents(documents, model_name="openai/text-embedding-3-small")
        index.set_index_id(title)
        index.storage_context.persist()
        shutil.rmtree("tmp")
    return index



def get_index_by_title(title):
    """
    Get an index by its title. If the index does not exist, returns None.

    Args:
        title (str): The title of the index

    Returns:
        VectorStoreIndex: The index with the specified title
    """
    try:
        storage_context = StorageContext.from_defaults(persist_dir="storage")
        index = load_index_from_storage(storage_context, index_id=title)
        return index
    except ValueError:
        return None

## Question Answering Functions


In [8]:
def answer_reading_comprehension(question, context_title=None, context="", use_rag=False):
    """
    Answer a question given a context. If use_rag is True, retrieval will be used to answer the question. Otherwise, the entire context will be prepended to the question.

    Args:
        question (str): The question to answer
        context_title (str): The title of the context index
        context (str): The context to use
        use_rag (bool): Whether to use retrieval to answer the question

    Returns:
        str: The answer to the question
    """
    if use_rag:
        if context_title is None:
            raise ValueError("context_title must be provided when using RAG")
        #index = create_index_from_text(context, context_title)
        index = create_index_from_text_with_ids(context, context_title, chunk_size=500)

        index = add_summarization_embeddings_with_ids(index, chunk_size=500, new_storage_location="storage_same_chunk_summary", index_title="summary_index")
        
        query_engine = index.as_query_engine()
        # Query the vector store to find top matching chunks
        top_chunks = query_engine.query(question, top_k=top_k)
        
        # Extract and combine the text from the top matching chunks for use as context
        top_chunks_text = ' '.join([chunk['text'] for chunk in top_chunks])

        raw_text_chunk_count =0
        summary_chunk_count = 0
        for chunk in top_chunks:
            if chunk['text'].startswith("text_chunk_"):
                raw_text_chunk_count += 1
            elif chunk['text'].startswith("summary_"):
                summary_chunk_count += 1

        print(f"Raw text chunk count: {raw_text_chunk_count}")
        print(f"Summary chunk count: {summary_chunk_count}")
        
        # Use the combined context of top chunks to generate an answer to the question
        # Construct a prompt that guides the LLM to consider the context and answer the question subjectively or conceptually
        prompt = f"Consider the following context with depth and thoughtfulness: {top_chunks_text}\n\nCan you respond to the following question with insight and nuance:?\n\nQuestion: {question}\n\nAnswer:"
        #combined_context = top_chunks_text + "\n" + question
        response = Settings.llm.complete(prompt).text

        # response = query_engine.query(question).response
    else:
        response = Settings.llm.complete(context + "\n" + question).text
    return response


def answer_reading_comprehension_with_rag(*args, **kwargs):
    """
    Answer a question given a context using retrieval to answer the question.

    Args:
        question (str): The question to answer
        context_title (str): The title of the context index
        context (str): The context to use

    Returns:
        str: The answer to the question
    """
    return answer_reading_comprehension(*args, **kwargs, use_rag=True)


def answer_reading_comprehension_in_context(*args, **kwargs):
    """
    Answer a question given a context. The context will be prepended to the question.

    Args:
        question (str): The question to answer
        context (str): The context to use

    Returns:
        str: The answer to the question
    """
    return answer_reading_comprehension(*args, **kwargs, use_rag=False)

## Evaluation


### Load the dataset and metrics


In [9]:
longdep_qa_ds = load_dataset("bigainlco/LooGLE", "longdep_qa", split="test")
rouge = evaluate.load("rouge")


def get_rouge_metrics(output_file):
    """
    Get ROUGE metrics for a .jsonl file containing generated answers and ground truth answers.

    Args:
        output_file (str): The path to the .jsonl file

    Returns:
        dict: The ROUGE metrics
    """
    with open(output_file, "r") as f:
        lines = f.readlines()
    outputs = [json.loads(line) for line in lines]
    generated_answers = [output["generated_answer"] for output in outputs]
    ground_truths = [output["ground_truth"] for output in outputs]
    rouge_metrics = rouge.compute(predictions=generated_answers, references=ground_truths)
    return rouge_metrics


def llm_self_score(output_file):
    """
    Score the generated answers in a .jsonl file using the LLM. The user will be prompted to determine whether each answer is correct.

    Args:
        output_file (str): The path to the .jsonl file

    Returns:
        float: The accuracy of the generated answers
    """
    with open(output_file, "r") as f:
        lines = f.readlines()
    outputs = [json.loads(line) for line in lines]
    llm = Settings.llm
    for output in outputs:
        question = output["question"]
        ground_truth = output["ground_truth"]
        generated_answer = output["generated_answer"]
        if "correct" in output:
            continue
        prompt = f'Given the question "{question}" whose answer is "{ground_truth}", is answer "{generated_answer}" similar enough to the true answer that it should be considered correct? Answer "yes" or "no" with no other characters or capitalization.'
        response = llm.complete(prompt).text.lower()
        if "yes" in response:
            output["correct"] = True
        else:
            output["correct"] = False
    num_correct = sum([output["correct"] for output in outputs])
    accuracy = num_correct / len(outputs)
    with open(output_file, "w") as f:
        for output in outputs:
            json.dump(output, f)
            f.write("\n")
    return accuracy

/Users/isaac/miniforge3/envs/LongRAG/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for bigainlco/LooGLE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigainlco/LooGLE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


### Inference function and helpers


In [41]:
def read_output_file(output_file):
    """
    Read a .jsonl file containing generated answers and ground truth answers.

    Args:
        output_file (str): The path to the .jsonl file

    Returns:
        list: The outputs in the file
    """
    if not os.path.exists(output_file):
        return []
    with open(output_file, "r") as f:
        lines = f.readlines()
    outputs = [json.loads(line) for line in lines]
    return outputs


def log_outputs(question, ground_truth, generated_answer, output_file):
    """
    Log a question, its ground truth answer, and a generated answer to a .jsonl file.

    Args:
        question (str): The question
        ground_truth (str): The ground truth answer
        generated_answer (str): The generated answer
        output_file (str): The path to the .jsonl file; if None, a new file will be created in the "output" directory with the current time as the name

    Returns:
        tuple: The path to the .jsonl file and the outputs in the file
    """
    if output_file is None:
        os.makedirs("output", exist_ok=True)
        output_file = f"output/{time.time()}.jsonl"
    with open(output_file, "a") as f:
        json.dump({"question": question, "ground_truth": ground_truth, "generated_answer": generated_answer}, f)
        f.write("\n")
    existing_output = read_output_file(output_file)
    return output_file, existing_output


def question_is_answered(question, existing_output):
    """
    Determine whether a question has already been answered in a list of outputs.

    Args:
        question (str): The question
        existing_output (list): The outputs

    Returns:
        bool: Whether the question has already been answered
    """
    if question in [output["question"] for output in existing_output]:
        return True
    return False


def test_longdep_qa(inference_function, output_file=None, debug_lim=None):
    """
    Test an inference function on the longdep_qa dataset.

    Args:
        inference_function (function): The function to test
        output_file (str): The path to the .jsonl file to log outputs to; if None, a new file will be created in the "output" directory with the current time as the name
        debug_lim (int): The number of questions to test; if None, all questions will be tested

    Returns:
        None
    """
    n_questions = sum([len(eval(env["qa_pairs"])) for env in longdep_qa_ds])
    if debug_lim is None:
        debug_lim = n_questions
    existing_output = read_output_file(output_file)
    with tqdm.tqdm(total=debug_lim) as pbar:
        for environment in longdep_qa_ds:
            context = environment["input"]
            title = environment["title"]
            qa_pairs = eval(environment["qa_pairs"])
            for question_dict in qa_pairs:
                question = question_dict["Q"]
                ground_truth = question_dict["A"]
                if not question_is_answered(question, existing_output):
                    generated_answer = inference_function(question, context_title=title, context=context)
                    output_file, existing_output = log_outputs(question, ground_truth, generated_answer, output_file)
                pbar.update(1)
                if pbar.n >= debug_lim:
                    break
            if pbar.n >= debug_lim:
                break

### Inference with RAG


In [35]:
test_longdep_qa(answer_reading_comprehension_with_rag, output_file="output/baseline_with_rag_100.jsonl", debug_lim=100)


100%|██████████| 100/100 [00:00<00:00, 22030.06it/s]


### Calculate metrics


In [63]:
rouge_metrics = get_rouge_metrics("output/baseline_with_rag_100.jsonl")
print("Rouge Metrics:", rouge_metrics)

llm_self_score = llm_self_score("output/baseline_with_rag_100.jsonl")
print("LLM Self-Score:", llm_self_score)

Rouge Metrics: {'rouge1': 0.19845643848785122, 'rouge2': 0.07862381462209828, 'rougeL': 0.1619569034833745, 'rougeLsum': 0.17183036257033626}
LLM Self-Score: 0.53
